In [3]:
# Import required packages 
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [40]:
# 读取文件
shop_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'ccf_first_round_shop_info.csv')
user_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'ccf_first_round_user_shop_behavior.csv')
eval_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'evaluation_public.csv')

shop_train = pd.read_csv(train_path, delimiter = ',')
user_train = pd.read_csv(user_path, delimiter = ',')
evaluation_data =  pd.read_csv(eval_path, delimiter = ',')
# landsat_test = pd.read_csv(test_path, delimiter = ',')

# 店铺和商场信息

In [43]:
shop_train.head(n=5)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


In [56]:
# 检测有多少个商场，以及他们的名字

unique_mall = []
idx = 0
for i in range(len(shop_train['mall_id'])):
    if shop_train['mall_id'][i] not in unique_mall:
        unique_mall.append(shop_train['mall_id'][i])
        idx += 1
print(unique_mall)
print(len(unique_mall))

['m_690', 'm_6587', 'm_5892', 'm_625', 'm_3839', 'm_3739', 'm_1293', 'm_1175', 'm_2182', 'm_2058', 'm_3871', 'm_3005', 'm_822', 'm_2467', 'm_4406', 'm_909', 'm_4923', 'm_2224', 'm_2333', 'm_4079', 'm_5085', 'm_2415', 'm_4543', 'm_7168', 'm_2123', 'm_4572', 'm_1790', 'm_3313', 'm_4459', 'm_1409', 'm_979', 'm_7973', 'm_1375', 'm_4011', 'm_1831', 'm_4495', 'm_1085', 'm_3445', 'm_626', 'm_8093', 'm_4828', 'm_6167', 'm_3112', 'm_4341', 'm_622', 'm_4422', 'm_2267', 'm_615', 'm_4121', 'm_9054', 'm_4515', 'm_1950', 'm_3425', 'm_3501', 'm_4548', 'm_5352', 'm_3832', 'm_1377', 'm_1621', 'm_1263', 'm_2578', 'm_2270', 'm_968', 'm_1089', 'm_7374', 'm_2009', 'm_6337', 'm_7601', 'm_623', 'm_5154', 'm_5529', 'm_4168', 'm_3916', 'm_2878', 'm_9068', 'm_3528', 'm_4033', 'm_3019', 'm_1920', 'm_8344', 'm_6803', 'm_3054', 'm_8379', 'm_1021', 'm_2907', 'm_4094', 'm_4187', 'm_5076', 'm_3517', 'm_2715', 'm_5810', 'm_5767', 'm_4759', 'm_5825', 'm_7994', 'm_7523', 'm_7800']
97


我们发现总共有97个商场，初步设想，可以数据集根据商场不同，分成97份，每份对应不同的店铺

In [33]:
shop_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8477 entries, 0 to 8476
Data columns (total 6 columns):
shop_id        8477 non-null object
category_id    8477 non-null object
longitude      8477 non-null float64
latitude       8477 non-null float64
price          8477 non-null int64
mall_id        8477 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 397.4+ KB


In [41]:
shop_train.describe()

,longitude,latitude,price
count,8477.000000,8477.000000,8477.000000
mean,117.426526,32.961908,50.527309
std,4.367982,4.687368,9.416713
min,104.414352,23.157355,24.000000
25%,114.829869,30.911022,43.000000
50%,118.192157,32.181703,50.000000
75%,120.927247,36.827750,59.000000
max,123.801593,42.367802,88.000000


# 用户在店铺内交易信息

In [29]:
user_train.head(n=5)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [36]:
user_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138015 entries, 0 to 1138014
Data columns (total 6 columns):
user_id       1138015 non-null object
shop_id       1138015 non-null object
time_stamp    1138015 non-null object
longitude     1138015 non-null float64
latitude      1138015 non-null float64
wifi_infos    1138015 non-null object
dtypes: float64(2), object(4)
memory usage: 52.1+ MB


In [42]:
user_train.describe()

,longitude,latitude
count,1.138015e+06,1.138015e+06
mean,1.174162e+02,3.286613e+01
std,4.359182e+00,4.392070e+00
min,-1.139504e+02,5.364270e-01
25%,1.146429e+02,3.097768e+01
50%,1.180478e+02,3.208677e+01
75%,1.210030e+02,3.525854e+01
max,1.271192e+02,5.110064e+01


# 预测集

In [31]:
evaluation_data.head(n=5)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [38]:
evaluation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483931 entries, 0 to 483930
Data columns (total 7 columns):
row_id        483931 non-null int64
user_id       483931 non-null object
mall_id       483931 non-null object
time_stamp    483931 non-null object
longitude     483931 non-null float64
latitude      483931 non-null float64
wifi_infos    483931 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 25.8+ MB


In [39]:
evaluation_data.describe()

,row_id,longitude,latitude
count,483931.000000,483931.000000,483931.000000
mean,241966.000000,117.520898,32.880096
std,139698.990897,4.236408,4.496274
min,1.000000,87.841705,22.766598
25%,120983.500000,114.831137,30.965195
50%,241966.000000,118.048544,31.975273
75%,362948.500000,121.002735,35.552129
max,483931.000000,127.119151,47.581047
